In [1]:
from sklearn.metrics import mean_squared_error
import time
import psycopg2
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
import pickle

In [ ]:
import psycopg2
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
import pickle

# Load the saved model and scaler
loaded_model = load_model('bitcoin_price_model.keras')
loaded_scaler = pickle.load(open('bitcoin_price_scaler.pkl', 'rb'))

# PostgreSQL connection configuration
conn = psycopg2.connect(
    "dbname='Bitcoin' user='postgres' host='localhost' password='root'"
)

# Function to create the "predicted_data" table in PostgreSQL
def create_predicted_data_table():
    try:
        cur = conn.cursor()
        cur.execute("""
            CREATE TABLE IF NOT EXISTS predicted_data (
                timestamp TIMESTAMP PRIMARY KEY,
                actual_value NUMERIC,
                predicted_value NUMERIC
            )
        """)
        conn.commit()
        print("Table 'predicted_data' created successfully.")
    except Exception as e:
        print("Error creating table 'predicted_data':", e)
        conn.rollback()

# Function to fetch live data from PostgreSQL
def fetch_live_data():
    try:
        cur = conn.cursor()
        cur.execute("SELECT * FROM public.bitcoin_data WHERE event_time >= '2024-05-26' ORDER BY event_time ASC")
        rows = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        df_live = pd.DataFrame(rows, columns=columns)
        return df_live
    except Exception as e:
        print("Error fetching live data:", e)
        conn.rollback()  # Rollback the transaction in case of error
        return None

# Function to preprocess live data
def preprocess_live_data(data, scaler):
    data['event_time'] = pd.to_datetime(data['event_time'])
    data.set_index('event_time', inplace=True)
    data['close_price_scaled'] = scaler.transform(data[['close_price']])
    return data

# Function to make predictions on live data
def make_predictions(model, live_data, scaler, window_size):
    X_live = live_data['close_price_scaled'].values.reshape(-1, 1)
    predictions = model.predict(X_live)
    predictions_actual = scaler.inverse_transform(predictions)
    return predictions_actual

# Function to insert predictions into PostgreSQL
def insert_predictions(timestamps, actual_values, predicted_values):
    try:
        cur = conn.cursor()
        for i in range(len(timestamps)):
            cur.execute("INSERT INTO predicted_data (timestamp, actual_value, predicted_value) VALUES (%s, %s, %s) ON CONFLICT (timestamp) DO UPDATE SET actual_value = EXCLUDED.actual_value, predicted_value = EXCLUDED.predicted_value", (timestamps[i], float(actual_values[i]), float(predicted_values[i][0])))
        conn.commit()
        print("Predictions inserted successfully into PostgreSQL.")
    except Exception as e:
        print("Error inserting predictions into PostgreSQL:", e)
        conn.rollback()

# Define the interval for fetching new data and making predictions (in seconds)
prediction_interval = 60  # Every minute

# Create the "predicted_data" table
create_predicted_data_table()

# Continuously fetch new data and make predictions
while True:
    live_data = fetch_live_data()

    if live_data is not None and 'event_time' in live_data.columns and 'close_price' in live_data.columns:
        live_data_preprocessed = preprocess_live_data(live_data, loaded_scaler)
        predictions = make_predictions(loaded_model, live_data_preprocessed, loaded_scaler, window_size=10)
        timestamps = live_data_preprocessed.index.tolist()
        actual_values = live_data['close_price'].tolist()

        insert_predictions(timestamps, actual_values, predictions)

    time.sleep(prediction_interval)

C:\Users\ajink\anaconda3\envs\Kafka\lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Table 'predicted_data' created successfully.
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
Predictions inserted successfully into PostgreSQL.
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Predictions inserted successfully into PostgreSQL.
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 970us/step
Predictions inserted successfully into PostgreSQL.
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Predictions inserted successfully into PostgreSQL.
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 887us/step
Predictions inserted successfully into PostgreSQL.
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step
Predictions inserted successfully into PostgreSQL.
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 986us/step
Predictions inserted successfully into PostgreSQL.
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Predictions inserted successfully into PostgreSQL.
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 979us/step
Predictions inserted successfully into PostgreSQL.
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Predictions inserted successfully into PostgreSQL.
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pre